In [96]:
import pandas as pd
import numpy as np

In [97]:
df_score = pd.read_csv("output/2026_visitors_pred.csv")

In [98]:
# --- 1. Snow Reliability Score ---
df_score["snow_score"] = (
    - 0.5 * df_score["avg_max_temp"]    # colder max temp = better
    - 0.5 * df_score["avg_min_temp"]  # weight min temp
    + 0.5 * df_score["sum_precip"]    # more snow = better
)

In [99]:
# --- 2. Normalize Crowding & Snow (0–1 scaling) ---
def minmax_norm(s):
    return (s - s.min()) / (s.max() - s.min())

df_score["snow_norm"] = minmax_norm(df_score["snow_score"])
df_score["crowd_norm"] = minmax_norm(df_score["visitors_pred"])  # higher crowd = worse

In [100]:
# --- 3. Price ---
ratio_df = pd.read_csv("../data/resort_sentiment_ratio.csv")

df_score = df_score.merge(ratio_df, on="Resort", how="left")

df_score.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             135 non-null    int64  
 1   Week             135 non-null    int64  
 2   Resort           135 non-null    object 
 3   avg_max_temp     135 non-null    float64
 4   avg_min_temp     135 non-null    float64
 5   sum_precip       135 non-null    float64
 6   visitors_pred    135 non-null    float64
 7   snow_score       135 non-null    float64
 8   snow_norm        135 non-null    float64
 9   crowd_norm       135 non-null    float64
 10  sentiment_ratio  135 non-null    float64
 11  season_price     135 non-null    float64
 12  day_price        135 non-null    float64
 13  state            135 non-null    object 
dtypes: float64(10), int64(2), object(2)
memory usage: 14.9+ KB


In [101]:
# --- Normalize prices ---
df_score["season_price_norm"] = (
    (df_score["season_price"] - df_score["season_price"].min()) /
    (df_score["season_price"].max() - df_score["season_price"].min())
)

df_score["day_price_norm"] = (
    (df_score["day_price"] - df_score["day_price"].min()) /
    (df_score["day_price"].max() - df_score["day_price"].min())
)

In [102]:
# Normalize sentiment
df_score["sentiment_norm"] = (
    (df_score["sentiment_ratio"] - df_score["sentiment_ratio"].min()) /
    (df_score["sentiment_ratio"].max() - df_score["sentiment_ratio"].min())
)

In [103]:
# --- Holiday Score ---
# weights can be tuned (α=0.5, β=0.3, γ=0.2 for now)
df_score["holiday_score_persona1"] = (
    0.45 * df_score["snow_norm"]
    - 0.3 * df_score["crowd_norm"]
    - 0.05 * df_score["season_price_norm"]
    + 0.2 * df_score["sentiment_norm"]
)

# weights can be tuned 
df_score["holiday_score_persona2"] = (
    0.25 * df_score["snow_norm"]
    - 0.2 * df_score["crowd_norm"]
    - 0.25 * df_score["day_price_norm"]
    + 0.3 * df_score["sentiment_norm"]
)


In [115]:
# --- 5. Rank ---
# --- VIC ranking ---
df_vic = df_score[df_score["state"]=="VIC"]
df_vic_rank_persona1 = df_vic.sort_values("holiday_score_persona1", ascending=False).reset_index(drop=True)
df_vic_rank_persona2 = df_vic.sort_values("holiday_score_persona2", ascending=False).reset_index(drop=True)

# --- NSW ranking ---
df_nsw = df_score[df_score["state"]=="NSW"]
df_nsw_rank_persona1 = df_nsw.sort_values("holiday_score_persona1", ascending=False).reset_index(drop=True)
df_nsw_rank_persona2 = df_nsw.sort_values("holiday_score_persona2", ascending=False).reset_index(drop=True)


In [121]:
df_vic_rank_persona1.head(15)

,Year,Week,Resort,avg_max_temp,avg_min_temp,sum_precip,visitors_pred,snow_score,snow_norm,crowd_norm,sentiment_ratio,season_price,day_price,state,season_price_norm,day_price_norm,sentiment_norm,holiday_score_persona1,holiday_score_persona2
0,2026,11,Mt. Stirling,3.402935,-1.523439,290.578079,1702.3113,144.349292,1.000000,0.027100,6.5,959.0,67.0,VIC,0.311969,0.000000,0.196429,0.465557,0.303509
1,2026,11,Mt. Baw Baw,3.402935,-1.523439,290.578079,8090.0600,144.349292,1.000000,0.128789,2.0,555.0,79.0,VIC,0.000000,0.084507,0.035714,0.418506,0.213830
2,2026,1,Mt. Buller,2.844732,-1.196906,264.021696,25941.0700,131.186936,0.820565,0.412968,29.0,1649.0,126.0,VIC,0.844788,0.415493,1.000000,0.403125,0.318674
3,2026,11,Mt. Buller,3.402935,-1.523439,290.578079,43546.2150,144.349292,1.000000,0.693232,29.0,1649.0,126.0,VIC,0.844788,0.415493,1.000000,0.399791,0.307480
4,2026,1,Mt. Stirling,2.844732,-1.196906,264.021696,2570.7961,131.186936,0.820565,0.040926,6.5,959.0,67.0,VIC,0.311969,0.000000,0.196429,0.380664,0.255885
5,2026,1,Mt. Baw Baw,2.844732,-1.196906,264.021696,7587.4307,131.186936,0.820565,0.120788,2.0,555.0,79.0,VIC,0.000000,0.084507,0.035714,0.340161,0.170571
6,2026,1,Mt. Hotham,2.844732,-1.196906,264.021696,13017.1520,131.186936,0.820565,0.207226,5.2,1379.0,186.0,VIC,0.636293,0.838028,0.150000,0.305272,-0.000811
7,2026,11,Mt. Hotham,3.402935,-1.523439,290.578079,31883.6930,144.349292,1.000000,0.507571,5.2,1379.0,186.0,VIC,0.636293,0.838028,0.150000,0.295914,-0.016021
8,2026,1,Falls Creek,2.844732,-1.196906,264.021696,13500.3060,131.186936,0.820565,0.214918,4.0,1379.0,177.0,VIC,0.636293,0.774648,0.107143,0.294393,0.000639
9,2026,15,Mt. Buller,6.523578,0.415611,210.955337,17541.1460,102.008074,0.422786,0.279246,29.0,1649.0,126.0,VIC,0.844788,0.415493,1.000000,0.264241,0.245974


In [118]:
df_vic_rank_persona2.head(5)

,Year,Week,Resort,avg_max_temp,avg_min_temp,sum_precip,visitors_pred,snow_score,snow_norm,crowd_norm,sentiment_ratio,season_price,day_price,state,season_price_norm,day_price_norm,sentiment_norm,holiday_score_persona1,holiday_score_persona2
0,2026,1,Mt. Buller,2.844732,-1.196906,264.021696,25941.0700,131.186936,0.820565,0.412968,29.0,1649.0,126.0,VIC,0.844788,0.415493,1.000000,0.403125,0.318674
1,2026,11,Mt. Buller,3.402935,-1.523439,290.578079,43546.2150,144.349292,1.000000,0.693232,29.0,1649.0,126.0,VIC,0.844788,0.415493,1.000000,0.399791,0.307480
2,2026,11,Mt. Stirling,3.402935,-1.523439,290.578079,1702.3113,144.349292,1.000000,0.027100,6.5,959.0,67.0,VIC,0.311969,0.000000,0.196429,0.465557,0.303509
3,2026,1,Mt. Stirling,2.844732,-1.196906,264.021696,2570.7961,131.186936,0.820565,0.040926,6.5,959.0,67.0,VIC,0.311969,0.000000,0.196429,0.380664,0.255885
4,2026,15,Mt. Buller,6.523578,0.415611,210.955337,17541.1460,102.008074,0.422786,0.279246,29.0,1649.0,126.0,VIC,0.844788,0.415493,1.000000,0.264241,0.245974


In [119]:
df_nsw_rank_persona1.head(5)

,Year,Week,Resort,avg_max_temp,avg_min_temp,sum_precip,visitors_pred,snow_score,snow_norm,crowd_norm,sentiment_ratio,season_price,day_price,state,season_price_norm,day_price_norm,sentiment_norm,holiday_score_persona1,holiday_score_persona2
0,2026,11,Selwyn,3.402935,-1.523439,290.578079,8911.5610,144.349292,1.000000,0.141867,1.000000,799.0,119.0,NSW,0.188417,0.366197,0.000000,0.398019,0.130077
1,2026,1,Selwyn,2.844732,-1.196906,264.021696,5809.8774,131.186936,0.820565,0.092490,1.000000,799.0,119.0,NSW,0.188417,0.366197,0.000000,0.332086,0.095094
2,2026,1,Charlotte Pass,2.541166,-2.107618,257.201319,15469.8190,128.383885,0.782353,0.246271,1.416667,1759.0,175.0,NSW,0.929730,0.760563,0.014881,0.234667,-0.039343
3,2026,1,Thredbo,2.844732,-1.196906,264.021696,23074.0230,131.186936,0.820565,0.367326,4.000000,1850.0,209.0,NSW,1.000000,1.000000,0.107143,0.230485,-0.086181
4,2026,11,Thredbo,3.402935,-1.523439,290.578079,46674.1370,144.349292,1.000000,0.743027,4.000000,1850.0,209.0,NSW,1.000000,1.000000,0.107143,0.198520,-0.116463


In [120]:
df_nsw_rank_persona2.head(5)

,Year,Week,Resort,avg_max_temp,avg_min_temp,sum_precip,visitors_pred,snow_score,snow_norm,crowd_norm,sentiment_ratio,season_price,day_price,state,season_price_norm,day_price_norm,sentiment_norm,holiday_score_persona1,holiday_score_persona2
0,2026,11,Selwyn,3.402935,-1.523439,290.578079,8911.5610,144.349292,1.000000,0.141867,1.0,799.0,119.0,NSW,0.188417,0.366197,0.0,0.398019,0.130077
1,2026,1,Selwyn,2.844732,-1.196906,264.021696,5809.8774,131.186936,0.820565,0.092490,1.0,799.0,119.0,NSW,0.188417,0.366197,0.0,0.332086,0.095094
2,2026,5,Selwyn,2.300878,-1.843534,218.327850,9653.2690,108.935253,0.517220,0.153675,1.0,799.0,119.0,NSW,0.188417,0.366197,0.0,0.177226,0.007021
3,2026,8,Selwyn,3.215796,-1.954643,212.438419,8401.7400,105.588633,0.471598,0.133751,1.0,799.0,119.0,NSW,0.188417,0.366197,0.0,0.162673,-0.000400
4,2026,15,Selwyn,6.523578,0.415611,210.955337,4775.4330,102.008074,0.422786,0.076022,1.0,799.0,119.0,NSW,0.188417,0.366197,0.0,0.158026,-0.001057


In [122]:
df_score.to_csv("output/2026_holiday_score_pred.csv", index=False)